In [1]:
# 导入所需要的库和模块
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

**注意：**
下面代码中 misformatted 这个示例代码块是错误的，因为它没有正确地格式化为 JSON。在实际应用中，你需要确保你的代码块是正确格式化的，e.g.,

```
```json
{
  "flower_type": "玫瑰",
  "price": 50,
  "description": "只需50元，即可拥有象征着热情与爱情的经典玫瑰。每一朵精选玫瑰，都是对美好情感的致敬，无论是浪漫的告白，还是温馨的节日祝福，它们都是传递心意的完美使者。",
  "reason": "这个描述旨在通过强调玫瑰的象征意义和亲民价格，激发消费者的购买欲望，同时突出其作为礼物的理想选择。"
}
```
```

In [2]:
# 使用Pydantic创建一个数据格式，表示花
class Flower(BaseModel):
    name: str = Field(description="name of a flower")
    colors: List[str] = Field(description="the colors of this flower")
# 定义一个用于获取某种花的颜色列表的查询
flower_query = "Generate the charaters for a random flower."

# 定义一个格式不正确的输出
misformatted = "{'name': '康乃馨', 'colors': ['粉红色','白色','红色','紫色','黄色']}"

# 创建一个用于解析输出的Pydantic解析器，此处希望解析为Flower格式
parser = PydanticOutputParser(pydantic_object=Flower)
# 使用Pydantic解析器解析不正确的输出
parser.parse(misformatted) # 这行代码会出错
# parser.parse(misformatted.replace("'", "\"")) # fix of the above line

OutputParserException: Invalid json output: {'name': '康乃馨', 'colors': ['粉红色','白色','红色','紫色','黄色']}

In [3]:
# 从langchain库导入所需的模块
from langchain_openai import AzureChatOpenAI
from langchain.output_parsers import OutputFixingParser
import os

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("BASE_URL"),
    openai_api_version=os.getenv("API_VERSION"),
    deployment_name=os.getenv("DEPLOYMENT_NAME"),
    openai_api_key=os.getenv("API_KEY"),
)

# 使用OutputFixingParser创建一个新的解析器，该解析器能够纠正格式不正确的输出
new_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

In [4]:
# 使用新的解析器解析不正确的输出
result = new_parser.parse(misformatted) # 错误被自动修正
print(result) # 打印解析后的输出结果

name='Carnation' colors=['pink', 'white', 'red', 'purple', 'yellow']
